## Crawl a page

In [ ]:
import pandas as pd
import re
import datetime
from pandas import DataFrame
import undetected_chromedriver as uc
from time import sleep
from selenium.common import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException
import urllib.request
import os
from dotenv import load_dotenv

In [4]:
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--window-size=1920x1080")
driver = uc.Chrome(options=options)

In [ ]:
driver.get('https://www.amazon.com/')

### Captcha

In [ ]:
from PIL import Image
from pytesseract import pytesseract

In [ ]:
captcha_image = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
    (By.CSS_SELECTOR, "body > div > div.a-row.a-spacing-double-large > div.a-section > div > div > form > div.a-row.a-spacing-large > div > div > div.a-row.a-text-center > img")))
image = captcha_image.get_attribute('src')
filename = datetime.datetime.now()
image_path = 'D:\CrawlDataAmazon\\'
folder_path = os.path.join(os.path.dirname(image_path), "data")
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
urllib.request.urlretrieve(image, os.path.join(folder_path, "{}.png".format(filename.strftime("%Y%m%d%H%M%S"))))

In [ ]:
if os.path.exists(folder_path):
    files = os.listdir(folder_path)
    if len(files) > 1:
        files.sort(key=lambda x: os.path.getmtime(os.path.join(folder_path, x)), reverse=True)
        for file in files[1:]:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
    print("List images:")
    for file in os.listdir(folder_path):
        print(file)
else:
    print("Folder not found.")

In [ ]:
path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
path_to_image = os.path.join(folder_path, "{}.png".format(filename.strftime("%Y%m%d%H%M%S")))
pytesseract.tesseract_cmd = path_to_tesseract
img = Image.open(path_to_image)

#Extract text from image
text = pytesseract.image_to_string(img)
print(text)

In [ ]:
captcha = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
    (By.CSS_SELECTOR, "#captchacharacters")))
captcha.send_keys(text)
sleep(2)

In [ ]:
submit = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
    (By.CSS_SELECTOR, "body > div > div.a-row.a-spacing-double-large > div.a-section > div > div > form > div.a-section.a-spacing-extra-large > div > span > span > button")))
submit.click()

### Crawl data

In [ ]:
search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
search_box.clear()

In [ ]:
# input = "gaming keyboard"

In [ ]:
keyword = input('What name do you want to crawl ?')

In [ ]:
search_box.send_keys(keyword)
search_button = driver.find_element(By.ID, 'nav-search-submit-button')
search_button.click()
driver.implicitly_wait(5)

In [ ]:
items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))

In [ ]:
product_name = []
product_code = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []

In [ ]:
for item in items:
    try:
        name = item.find_element(By.XPATH, './/span[@class="a-size-medium a-color-base a-text-normal"]')
        product_name.append(name.text)
    except:
        product_name.append("")
    
    try:
        data_code = item.get_attribute("data-asin")
        product_code.append(data_code)
    except:
        product_code.append("")
    
    # find price
    whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
    fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')
    
    if whole_price != [] and fraction_price != []:
        price = '.'.join([whole_price[0].text, fraction_price[0].text])
    else:
        price = 0
    product_price.append(price)

    # find ratings box
    ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

    # find ratings and ratings_num
    if ratings_box != []:
        ratings = ratings_box[0].get_attribute('aria-label')
        ratings_num = ratings_box[1].get_attribute('aria-label')
    else:
        ratings, ratings_num = 0, 0
    
    product_ratings.append(ratings)
    product_ratings_num.append(str(ratings_num))
    
    # find link
    link = item.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute("href")
    product_link.append(link)

In [ ]:
data = pd.DataFrame({'ProductId': product_code,'Name':product_name,
                         'Price': product_price, 'Link':product_link,'Rating':product_ratings,'Rating Count':product_ratings_num,
                         })

In [ ]:
data

## Crawl Multiple pages

In [1]:
import pandas as pd
import re
import datetime
from pandas import DataFrame
import undetected_chromedriver as uc
from time import sleep
from selenium.common import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException
import urllib.request
import os
from dotenv import load_dotenv
from PIL import Image
from pytesseract import pytesseract

In [8]:
options = uc.ChromeOptions()
options.add_argument("--start-maximized")
driver = uc.Chrome(options=options)
driver.get('https://www.amazon.com/')

### Captcha

In [11]:
count = 1
while count < 10:
    try:
        captcha_image = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "body > div > div.a-row.a-spacing-double-large > div.a-section > div > div > form > div.a-row.a-spacing-large > div > div > div.a-row.a-text-center > img")))
        image = captcha_image.get_attribute('src')
        filename = datetime.datetime.now()
        image_path = 'D:\CrawlDataAmazon\\'
        folder_path = os.path.join(os.path.dirname(image_path), "data")
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        urllib.request.urlretrieve(image, os.path.join(folder_path, "{}.png".format(filename.strftime("%Y%m%d%H%M%S"))))
        
        # check folder
        if os.path.exists(folder_path):
            files = os.listdir(folder_path)
            if len(files) > 1:
                files.sort(key=lambda x: os.path.getmtime(os.path.join(folder_path, x)), reverse=True)
                for file in files[1:]:
                    file_path = os.path.join(folder_path, file)
                    os.remove(file_path)
            print("List images:")
            for file in os.listdir(folder_path):
                print(file)
        else:
            print("Folder not found.")
        
        path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        path_to_image = os.path.join(folder_path, "{}.png".format(filename.strftime("%Y%m%d%H%M%S")))
        pytesseract.tesseract_cmd = path_to_tesseract
        img = Image.open(path_to_image)

        #Extract text from image
        text = pytesseract.image_to_string(img)
        print(text)
        
        captcha = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "#captchacharacters")))
        captcha.send_keys(text)
        sleep(2)
        
        submit = WebDriverWait(driver, 10).until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "body > div > div.a-row.a-spacing-double-large > div.a-section > div > div > form > div.a-section.a-spacing-extra-large > div > span > span > button")))
        submit.click()
        count +=1
    except TimeoutException:
        break

List images:
20230710153846.png

List images:
20230710153851.png

List images:
20230710153855.png

List images:
20230710153858.png

List images:
20230710153901.png

List images:
20230710153905.png
PBMPNR

List images:
20230710153909.png
CYALTPS

List images:
20230710153914.png
HI\XATU

List images:
20230710153917.png
HI\XATU



In [12]:
search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
search_box.clear()

In [ ]:
# input = "gaming keyboard"

In [13]:
keyword = input('What name do you want to crawl ?')

What name do you want to crawl ?gaming keyboard


In [14]:
search_box.send_keys(keyword)
search_button = driver.find_element(By.ID, 'nav-search-submit-button')
search_button.click()
driver.implicitly_wait(2)

In [15]:
product_name = []
product_code = []
product_price = []
product_ratings = []
product_ratings_num = []
product_link = []

### Crawl end of page

In [ ]:
# count = 1
# while True:
#     try:
#         print(f'Page {count}')
#         items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located(
#         (By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
#         for item in items:
#             try:
#                 name = item.find_element(By.XPATH, './/span[@class="a-size-medium a-color-base a-text-normal"]')
#                 product_name.append(name.text)
#             except:
#                 product_name.append("")

#             try:
#                 data_code = item.get_attribute("data-asin")
#                 product_code.append(data_code)
#             except:
#                 product_code.append("")

#             # find price
#             whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
#             fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')

#             if whole_price != [] and fraction_price != []:
#                 price = '.'.join([whole_price[0].text, fraction_price[0].text])
#             else:
#                 price = 0
#             product_price.append(price)

#             # find ratings box
#             ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

#             # find ratings and ratings_num
#             if ratings_box != []:
#                 ratings = ratings_box[0].get_attribute('aria-label')
#                 ratings_num = ratings_box[1].get_attribute('aria-label')
#             else:
#                 ratings, ratings_num = 0, 0

#             product_ratings.append(ratings)
#             product_ratings_num.append(str(ratings_num))

#             # find link
#             link = item.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute("href")
#             product_link.append(link)
#         next_page = driver.find_element(By.XPATH, './/a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
#         count +=1
#         next_page.click()
#         sleep(2)
#     except ElementNotInteractableException:
#         print('Element Not Interactable Exception')
#         break

### Crawl 4 pages

In [16]:
for i in range(4):
    try:
        print(f'Page {i}')
        items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located(
        (By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))
        for item in items:
            try:
                name = item.find_element(By.XPATH, './/span[@class="a-size-medium a-color-base a-text-normal"]')
                product_name.append(name.text)
            except:
                product_name.append("")

            try:
                data_code = item.get_attribute("data-asin")
                product_code.append(data_code)
            except:
                product_code.append("")

            # find price
            whole_price = item.find_elements(By.XPATH, './/span[@class="a-price-whole"]')
            fraction_price = item.find_elements(By.XPATH, './/span[@class="a-price-fraction"]')

            if whole_price != [] and fraction_price != []:
                price = '.'.join([whole_price[0].text, fraction_price[0].text])
            else:
                price = 0
            product_price.append(price)

            # find ratings box
            ratings_box = item.find_elements(By.XPATH, './/div[@class="a-row a-size-small"]/span')

            # find ratings and ratings_num
            if ratings_box != []:
                ratings = ratings_box[0].get_attribute('aria-label')
                ratings_num = ratings_box[1].get_attribute('aria-label')
            else:
                ratings, ratings_num = 0, 0

            product_ratings.append(ratings)
            product_ratings_num.append(str(ratings_num))

            # find link
            link = item.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute("href")
            product_link.append(link)
        next_page = driver.find_element(By.XPATH, './/a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
        count +=1
        next_page.click()
        sleep(2)
    except ElementNotInteractableException:
        print('Element Not Interactable Exception')
        break

Page 0
Page 1
Page 2
Page 3


In [17]:
data = pd.DataFrame({'ProductId': product_code,'Name':product_name,
                         'Price': product_price, 'Link':product_link,'Rating':product_ratings,'Rating Count':product_ratings_num,
                         })

In [18]:
data

,ProductId,Name,Price,Link,Rating,Rating Count
0,B0BSKX8W3B,ASUS ROG Azoth 75% Wireless DIY Custom Gaming ...,222.32,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,4.6 out of 5 stars,72
1,B0BF67DM6K,New SteelSeries Apex Pro TKL Wireless 2023 Ed....,249.99,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,4.0 out of 5 stars,88
2,B07ZGDPT4M,SteelSeries Apex 3 RGB Gaming Keyboard – 10-Zo...,24.98,https://www.amazon.com/SteelSeries-Apex-RGB-Ga...,4.7 out of 5 stars,"9,796"
3,B09X6GJ691,Razer Ornata V3 X Gaming Keyboard: Low-Profile...,34.98,https://www.amazon.com/Razer-Ornata-Gaming-Key...,4.7 out of 5 stars,"11,137"
4,B07QQB9VCV,"Logitech G PRO Mechanical Gaming Keyboard, Ult...",0,https://www.amazon.com/Logitech-Mechanical-Ten...,4.7 out of 5 stars,"4,816"
5,B09ZVLBP4M,Corsair K55 PRO LITE RGB Wired Membrane Gaming...,29.99,https://www.amazon.com/Corsair-Backlighting-In...,4.6 out of 5 stars,"13,953"
6,B0BV4BC7LV,Razer BlackWidow V4 Pro Wired Mechanical Gamin...,229.99,https://www.amazon.com/Razer-BlackWidow-Mechan...,4.5 out of 5 stars,115
7,B09NMM4FY6,Corsair K70 RGB PRO Mechanical Gaming Keyboard...,119.99,https://www.amazon.com/Corsair-Mechanical-Gami...,4.5 out of 5 stars,20
8,B0BM2YKK7F,Razer DeathStalker V2 Pro Wireless Gaming Keyb...,249.99,https://www.amazon.com/Razer-DeathStalker-Wire...,4.3 out of 5 stars,286
9,B0BQ7DLR2V,ROCCAT Magma Mini - 60% RGB Gaming Keyboard wi...,34.99,https://www.amazon.com/ROCCAT-Magma-Mini-Progr...,4.6 out of 5 stars,655
